In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [43]:
# Load the processed data
df = pd.read_csv(r'D:\Finplat\data\processed\AAPL_processed_20250316.csv', index_col='Date', parse_dates=True)
df.head(3)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,SMA_20,SMA_50,SMA_200,...,VIX_Correlation,VIX_RelativeStrength,Avg_Market_Correlation,PE_Ratio,EPS,ROE,Sentiment_Score,News_Volume,Sentiment_Relevance,Weighted_Sentiment
Date,,,,,,,,,,,,,,,,,,,,,
2023-03-15 00:00:00-04:00,149.680508,151.719938,148.423184,151.462540,77167900,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,80.565181,1.88,0.000133,0.0,0,0.0,0.0
2023-03-16 00:00:00-04:00,150.640845,154.897917,150.126032,154.294006,76161100,0.0,0.0,NaN,NaN,NaN,...,NaN,0.139199,NaN,82.071280,1.88,0.000133,0.0,0,0.0,0.0
2023-03-17 00:00:00-04:00,154.521703,155.175117,152.739671,153.452484,98944600,0.0,0.0,NaN,NaN,NaN,...,NaN,-0.115067,NaN,81.623662,1.88,0.000133,0.0,0,0.0,0.0


In [44]:
# Obtain columns with null values and the count of null values
null_columns = df.columns[df.isnull().any()]
print(df[null_columns].isnull().sum())

SMA_20                        19
SMA_50                        49
SMA_200                      199
RSI_14                        13
ROC_10                        10
BB_Middle                     19
BB_Std                        19
BB_Upper                      19
BB_Lower                      19
ATR_14                        13
S&P500_Return                  1
S&P500_Correlation            30
S&P500_RelativeStrength        1
Beta_S&P500                   60
NASDAQ_Return                  1
NASDAQ_Correlation            30
NASDAQ_RelativeStrength        1
Beta_NASDAQ                   60
DowJones_Return                1
DowJones_Correlation          30
DowJones_RelativeStrength      1
Beta_DowJones                 60
Gold_Return                    1
Gold_Correlation              30
Gold_RelativeStrength          1
Oil_Return                     1
Oil_Correlation               30
Oil_RelativeStrength           1
VIX_Return                     1
VIX_Correlation               30
VIX_Relati

Handling Missing Values:
1. SMA columns: The first n rows of SMAn will be NaN.The idea to handle them is to create an adaptive moving average.
                For rows < 20, just use the closing price. For rows between 20 and 50, use SMA20.
                For rows between 50 and 200, use SMA50, and beyond that use SMA200.

In [45]:
def create_adaptive_ma(df):
    # Start with a column filled with the closing price
    df['MA'] = df['Close'].copy()
    
    # Update values as moving averages become available
    # For days where SMA_20 is available
    mask_sma20 = df['SMA_20'].notna()
    df.loc[mask_sma20, 'MA'] = df.loc[mask_sma20, 'SMA_20']
    
    # For days where SMA_50 is available, override with SMA_50
    mask_sma50 = df['SMA_50'].notna()
    df.loc[mask_sma50, 'MA'] = df.loc[mask_sma50, 'SMA_50']
    
    # For days where SMA_200 is available, override with SMA_200
    mask_sma200 = df['SMA_200'].notna()
    df.loc[mask_sma200, 'MA'] = df.loc[mask_sma200, 'SMA_200']
    
    return df

df = create_adaptive_ma(df)
df.head(1)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,SMA_20,SMA_50,SMA_200,...,VIX_RelativeStrength,Avg_Market_Correlation,PE_Ratio,EPS,ROE,Sentiment_Score,News_Volume,Sentiment_Relevance,Weighted_Sentiment,MA
Date,,,,,,,,,,,,,,,,,,,,,
2023-03-15 00:00:00-04:00,149.680508,151.719938,148.423184,151.46254,77167900,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,80.565181,1.88,0.000133,0.0,0,0.0,0.0,151.46254


In [46]:
# Drop SMA columns
df.drop(columns=['SMA_20', 'SMA_50', 'SMA_200'], inplace=True)

2. Handling the RSI_14 and ROC_10 features: Missing values in RSI_14 can be filled with a neutral value of 50.
                                            Missing values in ROC_10 can be filled with no change assumption and fill in 0.

In [47]:
df['RSI_14'] = df['RSI_14'].fillna(50)
df['ROC_10'] = df['ROC_10'].fillna(0)

3. Handling the BB Columns: Using a similar adaptive method like SMAs.

In [48]:
# Start with the price for BB_Middle
df['Adaptive_BB_Middle'] = df['Close'].copy()

# Once BB_Middle is available, use that
mask = df['BB_Middle'].notna()
df.loc[mask, 'Adaptive_BB_Middle'] = df.loc[mask, 'BB_Middle']

# For standard deviation, start with a small fixed value (e.g., 1% of price)
df['Adaptive_BB_Std'] = df['Close'] * 0.01  # 1% of price as initial volatility

# Once actual BB_Std is available, use that
mask = df['BB_Std'].notna()
df.loc[mask, 'Adaptive_BB_Std'] = df.loc[mask, 'BB_Std']

# Calculate adaptive bands
df['Adaptive_BB_Upper'] = df['Adaptive_BB_Middle'] + (df['Adaptive_BB_Std'] * 2)
df['Adaptive_BB_Lower'] = df['Adaptive_BB_Middle'] - (df['Adaptive_BB_Std'] * 2)

# Normalized BB width (volatility measure)
df['BB_Width'] = (df['Adaptive_BB_Upper'] - df['Adaptive_BB_Lower']) / df['Adaptive_BB_Middle']

In [49]:
# Drop BB columns
df.drop(columns=['BB_Middle', 'BB_Std', 'BB_Upper','BB_Lower'], inplace=True)

4. Handling AIR_14 column: Adaptive AIR

In [50]:
# Calculate True Range (TR) first

# Fill NaN values in the shifted column first
shifted_close = df['Close'].shift(1)
shifted_close.iloc[0] = df['Close'].iloc[0]  # Just use the first day's close

high_low = df['High'] - df['Low']
high_close = (df['High'] - shifted_close).abs()
low_close = (df['Low'] - shifted_close).abs()
tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)


# First row of TR will be NaN because of the shift operation
# Fill it with the day's high-low range
#Get the first row's index value properly
first_index = df.index[0]

df['TR'] = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
first_index = df.index[0]
df.loc[first_index, 'TR'] = df.loc[first_index, 'High'] - df.loc[first_index, 'Low']

# 2. Fix the ATR calculation using the correct indexing
df['Adaptive_ATR'] = df['TR'].copy()  # Start with TR

# Now use that index value to set the TR for the first row
df.loc[first_index, 'TR'] = df.loc[first_index, 'High'] - df.loc[first_index, 'Low']


for i, (idx, _) in enumerate(df.iterrows()):
    if i > 0:  # Skip first row
        lookback = min(i+1, 14)
        start_idx = i-lookback+1
        df.loc[idx, 'Adaptive_ATR'] = df['TR'].iloc[start_idx:i+1].mean()


# Once ATR_14 is available, use that
mask = df['ATR_14'].notna()
df.loc[mask, 'Adaptive_ATR'] = df.loc[mask, 'ATR_14']

# Normalize ATR as percentage of price
df['Adaptive_ATR_Pct'] = df['Adaptive_ATR'] / df['Close'] * 100


# Drop ATR column
df.drop('ATR_14', axis = 1, inplace=True)

5. Handling SP features.

In [51]:
df['S&P500_Price'] = df['S&P500_Price'].fillna(method='ffill')
df['S&P500_Return'] = df['S&P500_Return'].fillna(0)
# Forward fill (use previous valid value)
df['S&P500_Correlation'] = df['S&P500_Correlation'].fillna(method='ffill')

# If missing at the start, then backfill those initial points
df['S&P500_Correlation'] = df['S&P500_Correlation'].fillna(method='bfill')

In [52]:
# Forward Fill
df['Beta_S&P500'] = df['Beta_S&P500'].fillna(method='ffill')
sector_avg_beta = 1.2 
df['Beta_S&P500'] = df['Beta_S&P500'].fillna(sector_avg_beta)

In [53]:
# Recalculate all relative strength values to ensure consistency
df['S&P500_RelativeStrength'] = df['Close'] / df['S&P500_Price']

6. Approach is same for Dow Jones and NASDAQ features.

In [54]:
df['NASDAQ_Return'] = df['NASDAQ_Return'].fillna(0)
# Forward fill (use previous valid value)
df['NASDAQ_Correlation'] = df['NASDAQ_Correlation'].fillna(method='ffill')

# If missing at the start, then backfill those initial points
df['NASDAQ_Correlation'] = df['NASDAQ_Correlation'].fillna(method='bfill')

# Forward Fill
df['Beta_NASDAQ'] = df['Beta_NASDAQ'].fillna(method='ffill')
sector_avg_beta = 1.2 
df['Beta_NASDAQ'] = df['Beta_NASDAQ'].fillna(sector_avg_beta)

df['NASDAQ_RelativeStrength'] = df['Close'] / df['NASDAQ_Price']

In [55]:
# List of all market indices/assets you're tracking
markets = ['DowJones', 'Gold', 'Oil', 'VIX']

# Process each market's columns
for market in markets:
    # Handle Price columns (forward fill is most appropriate)
    df[f'{market}_Price'] = df[f'{market}_Price'].fillna(method='ffill')
    
    # Handle Return columns (can be recalculated from prices if needed)
    missing_returns = df[f'{market}_Return'].isna()
    if missing_returns.any() and not df[f'{market}_Price'].isna().all():
        # Calculate returns for missing data points if we have the prices
        temp_returns = df[f'{market}_Price'].pct_change()
        df.loc[missing_returns, f'{market}_Return'] = temp_returns[missing_returns]
    # Fill any remaining NaNs with zeros or forward fill
    df[f'{market}_Return'] = df[f'{market}_Return'].fillna(0)
    
    # Handle Correlation (forward fill is appropriate)
    df[f'{market}_Correlation'] = df[f'{market}_Correlation'].fillna(method='ffill')
    # If missing at the start, then backfill those initial points
    df[f'{market}_Correlation'] = df[f'{market}_Correlation'].fillna(method='bfill')
    # If missing at the start, then backfill those initial points
    # Handle RelativeStrength (recalculate where possible)
    missing_rs = df[f'{market}_RelativeStrength'].isna()
    if missing_rs.any() and not df[f'{market}_Price'].isna().all():
        df.loc[missing_rs, f'{market}_RelativeStrength'] = df.loc[missing_rs, 'Close'] / df.loc[missing_rs, f'{market}_Price']
    # Forward fill any remaining
    df[f'{market}_RelativeStrength'] = df[f'{market}_RelativeStrength'].fillna(method='ffill')
    

In [56]:
 #Forward Fill
df['Beta_DowJones'] = df['Beta_DowJones'].fillna(method='ffill')
sector_avg_beta = 1.2 
df['Beta_DowJones'] = df['Beta_DowJones'].fillna(sector_avg_beta)

In [57]:
# Forward fill (use previous valid value)
df['Avg_Market_Correlation'] = df['Avg_Market_Correlation'].fillna(method='ffill')

# If missing at the start, then backfill those initial points
df['Avg_Market_Correlation'] = df['Avg_Market_Correlation'].fillna(method='bfill')

In [58]:
# Recheck for missing values
null_columns = df.columns[df.isnull().any()]
print(df[null_columns].isnull().sum())

Series([], dtype: float64)


In [59]:
len(df.columns)

58

In [60]:
# Make a csv file to store the modified dataframe
output_path = 'D:\\Finplat\\data\\processed\\AAPL_processed_clean.csv'
df.to_csv(output_path, 
          index=True,           # Keep the date index
          float_format='%.4f',  # Limit decimal places for floats
          encoding='utf-8')     # Specify encoding

print(f"Cleaned data saved to {output_path}")

Cleaned data saved to D:\Finplat\data\processed\AAPL_processed_clean.csv
